In [26]:
import pickle
import pandas as pd
import numpy as np

In [27]:
from scipy.sparse import csr_matrix
lil = pickle.load(open('lil.pkl', 'rb'))
csr  = csr_matrix(lil).astype(np.float32)
target = pd.read_csv('target.csv')['target']

In [28]:
print(lil.shape)
print(lil.nnz)

(48444, 300054)
9045387


In [34]:

import numpy as np
from math import sqrt
import lightgbm as lgb
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


def get_oof(clf, x_train, y, x_test):
    train_h, train_w = x_train.shape
    test_h, test_w = x_test.shape
    NFOLDS=5
    SEED=71
    kf = KFold(train_h, n_folds=NFOLDS, shuffle=True, random_state=SEED)
    oof_train = np.zeros((train_h,))
    oof_test = np.zeros((test_h,))
    oof_test_skf = np.empty((NFOLDS, test_h))
    lgbm_params =  {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        # 'max_depth': 15,
        'num_leaves': 100,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.75,
        'bagging_freq': 4,
        'learning_rate': 0.016,
        #'max_bin':1023,
        'verbose': 0
    }
    for i, (train_index, test_index) in enumerate(kf):
        print('\nFold {}'.format(i))
        x_tr = x_train[train_index]
        y_tr = y[train_index]
        y_te = y[test_index]
        x_te = x_train[test_index]
        lgtrain = lgb.Dataset(x_tr, y_tr)
                    #feature_name=x_train.columns.tolist())
        lgvalid = lgb.Dataset(x_te, y_te)
                    #feature_name=x_train.columns.tolist())
                    #categorical_feature = categorical)
        lgb_clf = lgb.train(
            lgbm_params,
            lgtrain,
            num_boost_round=2000,
            valid_sets=[lgtrain, lgvalid],
            valid_names=['train','valid'],
            early_stopping_rounds=50,
            verbose_eval=50
        )
        oof_train[test_index] = lgb_clf.predict(x_te)
        oof_test_skf[i, :]    = lgb_clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train, oof_test
#Ridge oof method from Faron's kernel
#I was using this to analyze my vectorization, but figured it would be interesting to add the results back into the dataset
#It doesn't really add much to the score, but it does help lightgbm converge faster
oof_train, oof_test = get_oof(None, csr, target, csr)
rms = sqrt(mean_squared_error(target, oof_train))
print('LGB OOF RMSE: {}'.format(rms))
mae = sqrt(mean_absolute_error(target, oof_train))
print('LGB OOF MAE: {}'.format(mae))
print("Modeling Stage")



Fold 0
Training until validation scores don't improve for 50 rounds.
[50]	train's rmse: 3.1727	valid's rmse: 2.72324
[100]	train's rmse: 2.99845	valid's rmse: 2.69233
[150]	train's rmse: 2.87179	valid's rmse: 2.68914
[200]	train's rmse: 2.76506	valid's rmse: 2.68941
Early stopping, best iteration is:
[181]	train's rmse: 2.80125	valid's rmse: 2.68648

Fold 1
Training until validation scores don't improve for 50 rounds.
[50]	train's rmse: 3.0511	valid's rmse: 3.27362
[100]	train's rmse: 2.89026	valid's rmse: 3.22215
[150]	train's rmse: 2.76895	valid's rmse: 3.20104
[200]	train's rmse: 2.67304	valid's rmse: 3.1914
[250]	train's rmse: 2.58367	valid's rmse: 3.19224
Early stopping, best iteration is:
[207]	train's rmse: 2.66066	valid's rmse: 3.18994

Fold 2
Training until validation scores don't improve for 50 rounds.
[50]	train's rmse: 3.23769	valid's rmse: 2.31007
[100]	train's rmse: 3.05482	valid's rmse: 2.27813
[150]	train's rmse: 2.92565	valid's rmse: 2.28158
Early stopping, best itera